<a href="https://colab.research.google.com/github/TaeGuSeo/Terms-and-Conditions-Analysis-System/blob/main/%ED%83%80%EC%9D%B4%ED%8B%80_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install opencv-python pytesseract scikit-learn pandas
!pip install konlpy  # KoNLPy 설치

import os
import cv2
import pytesseract
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Kkma  # Kkma 추가

!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-kor

import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"  # Tesseract 바이너리 파일 경로로 설정

# Kkma 형태소 분석기 초기화
kkma = Kkma()

# 한국어 형태소 분석 및 명사 추출 함수 정의
def extract_nouns(text):
    nouns = kkma.nouns(text)  # 명사 추출
    return " ".join(nouns)    # 추출된 명사를 공백으로 연결하여 반환

# 약관 설명 XML 파일들을 읽어서 데이터프레임으로 만들어주는 함수
def read_xml_files(xml_data_folder):
    data = []
    for filename in os.listdir(xml_data_folder):
        if filename.endswith(".xml"):
            filepath = os.path.join(xml_data_folder, filename)
            with open(filepath, 'r') as file:
                xml_text = file.read()
            data.append({'filename': filename, 'xml_text': xml_text})
    return pd.DataFrame(data)

# 이미지에서 텍스트 추출을 위한 함수
def extract_text_from_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang='kor')
    return text

# 법률 데이터셋과 유사도를 측정하는 함수
def find_most_similar_file(user_text, legal_data):
    tfidf_vectorizer = TfidfVectorizer(analyzer=extract_nouns, stop_words=['보험', '약관', '부분','기타'])
    tfidf_matrix = tfidf_vectorizer.fit_transform(legal_data['xml_text'])
    user_tfidf = tfidf_vectorizer.transform([user_text])

    similarities = cosine_similarity(user_tfidf, tfidf_matrix)
    most_similar_index = similarities.argmax()
    most_similar_filename = legal_data.loc[most_similar_index, 'filename']
    return most_similar_filename

if __name__ == "__main__":
    # 약관 설명 XML 파일들을 읽어서 데이터프레임으로 만듦
    xml_data_folder = "/content/drive/MyDrive/데이터 샘플/sample/약관/원천데이터"
    user_image_path = "/content/drive/MyDrive/데이터 샘플/20230801_044925.jpg"

    legal_data = read_xml_files(xml_data_folder)

    # 사용자가 입력한 이미지로부터 텍스트를 추출.
    user_text = extract_text_from_image(user_image_path)

    # 유사도 측정하여 가장 유사한 약관 파일 찾음
    most_similar_filename = find_most_similar_file(user_text, legal_data)

    # 파일 이름에서 앞의 3글자와 뒤의 6글자를 제외한 부분 추출
    keyword = most_similar_filename[4:-10]

    # 출력
    print("약관명:", keyword)